In [ ]:
!pip install transformers datasets evaluate torch

In [ ]:
import os
import pandas as pd
from google.colab import files
import io

In [ ]:
fname_train = os.path.join("drive/MyDrive/practicum/la_train.csv")
df_train = pd.read_csv(fname_train)

In [ ]:
fname_test = os.path.join("drive/MyDrive/practicum/la_test.csv")
df_test = pd.read_csv(fname_test)

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from datasets import load_dataset

dataset = load_dataset('csv',
                       data_files = {
                           'train': fname_train,
                           'test': fname_test
                       }
                    )

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-1c0324f6eae8cc30/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

BERT

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/11262 [00:00<?, ? examples/s]

Map:   0%|          | 0/2816 [00:00<?, ? examples/s]

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=18)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [ ]:


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.113325,0.725000
2,No log,0.887120,0.786000


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.113325,0.725000
2,No log,0.887120,0.786000
3,No log,0.811805,0.802000


TrainOutput(global_step=375, training_loss=0.9716997884114583, metrics={'train_runtime': 374.1166, 'train_samples_per_second': 8.019, 'train_steps_per_second': 1.002, 'total_flos': 789446559744000.0, 'train_loss': 0.9716997884114583, 'epoch': 3.0})

In [ ]:
training_args2 = TrainingArguments(
    output_dir="test_trainer", 
    evaluation_strategy="epoch", 
    warmup_steps=200)

In [ ]:
trainer2 = Trainer(
    model=model,
    args=training_args2,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer2.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.989556,0.760000
2,No log,0.858459,0.796000
3,No log,0.858459,0.796000


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=375, training_loss=0.72990673828125, metrics={'train_runtime': 349.6285, 'train_samples_per_second': 8.581, 'train_steps_per_second': 1.073, 'total_flos': 1052595412992000.0, 'train_loss': 0.72990673828125, 'epoch': 3.0})

In [ ]:
training_args3 = TrainingArguments(
    output_dir="test_trainer", 
    evaluation_strategy="epoch", 
    weight_decay=0.01)

In [ ]:
trainer3 = Trainer(
    model=model,
    args=training_args3,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer3.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.907331,0.795000
2,No log,0.934058,0.808000
3,No log,0.946056,0.820000


TrainOutput(global_step=375, training_loss=0.2812082316080729, metrics={'train_runtime': 350.1796, 'train_samples_per_second': 8.567, 'train_steps_per_second': 1.071, 'total_flos': 789446559744000.0, 'train_loss': 0.2812082316080729, 'epoch': 3.0})

In [ ]:
trainer3.evaluate(
)

{'eval_loss': 0.9460556507110596,
 'eval_accuracy': 0.82,
 'eval_runtime': 33.0259,
 'eval_samples_per_second': 30.279,
 'eval_steps_per_second': 3.785,
 'epoch': 3.0}

In [ ]:
!pip install optuna

In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=18)

In [ ]:
args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

trainer_search = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
best_run = trainer_search.hyperparameter_search(n_trials=10, direction="maximize")

DistilBERT

In [ ]:
# tokenize and split into training and testing sets

checkpoint = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length')

tokenized_datasets = dataset.map(tokenize_function, batched=True)

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(3000))
small_test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
distilbert_model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=18)

training_args = TrainingArguments(
    output_dir="test_trainer", 
    evaluation_strategy="epoch")

trainer_distilbert_basic = Trainer(
    model=distilbert_model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_test_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer_distilbert_basic.train()

In [ ]:
trainer_distilbert_basic.evaluate()

In [ ]:
training_args_manual = TrainingArguments(
    output_dir="test_trainer", 
    evaluation_strategy="epoch", 
    weight_decay=0.01,
    warmup_steps=200)

trainer_distilbert = Trainer(
    model=distilbert_model,
    args=training_args_manual,
    train_dataset=small_train_dataset,
    eval_dataset=small_test_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer_distilbert.train()

In [ ]:
trainer_distilbert.evaluate()

Longformer

In [ ]:
# tokenize and split into training and testing sets

checkpoint = "allenai/longformer-base-4096"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length')

tokenized_datasets = dataset.map(tokenize_function, batched=True)

small_train_dataset_long = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_test_dataset_long = tokenized_datasets["test"].shuffle(seed=42).select(range(400))

In [ ]:
longformer_model = AutoModelForSequenceClassification.from_pretrained('allenai/longformer-base-4096', num_labels=18, gradient_checkpointing=False)

training_args_manual = TrainingArguments(
    output_dir="test_trainer", 
    evaluation_strategy="epoch", 
    weight_decay=0.01,
    warmup_steps=200)

longformer_trainer = Trainer(
    model=longformer_model,
    args=training_args_manual,
    train_dataset=small_train_dataset_long,
    eval_dataset=small_test_dataset_long,
    compute_metrics=compute_metrics,
)

In [ ]:
longformer_trainer.train()

In [ ]:
longformer_trainer.evaluate()

In [ ]:
training_args = TrainingArguments(
    output_dir="test_trainer", 
    evaluation_strategy="epoch")

longformer_trainer2 = Trainer(
    model=longformer_model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_test_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
longformer_trainer2.train()

In [ ]:
longformer_trainer2.evaluate()

Hyperparameter Search

In [ ]:
import torch
from ray import tune
from ray.tune.schedulers import PopulationBasedTraining
from ray.tune import uniform
from random import randint

scheduler = PopulationBasedTraining(
    mode = "max",
    metric='objective',
    perturbation_interval=2,
    hyperparam_mutations={
        "weight_decay": lambda: uniform(0.0, 0.3),
        "learning_rate": lambda: uniform(1e-5, 5e-5),
        "per_gpu_train_batch_size": [8, 16, 32],
        "num_train_epochs": [2,3,4],
        "warmup_steps":lambda: randint(0, 500)
    }
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def model_init():
    return AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=18).to(device)

training_args = TrainingArguments(
    "test", evaluation_strategy="steps", eval_steps=500, disable_tqdm=True)

trainer = Trainer(
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    model_init=model_init,
    compute_metrics=compute_metrics,
)

best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="ray",
    n_trials=4,
    keep_checkpoints_num=1,
    scheduler=scheduler)

In [ ]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model()